In [1]:

import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

In [2]:

def create_model(input_shape, pool_size):
    # Create the actual neural network here
    model = Sequential()
    # Normalizes incoming inputs. First layer needs the input shape to work
    model.add(BatchNormalization(input_shape=input_shape))

    # Below layers were re-named for easier reading of model summary; this not necessary
    # Conv Layer 1
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Conv1'))

    # Conv Layer 2
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Conv2'))

    # Pooling 1
    model.add(MaxPooling2D(pool_size=pool_size))

    # Conv Layer 3
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Conv3'))
    model.add(Dropout(0.2))

    # Conv Layer 4
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Conv4'))
    model.add(Dropout(0.2))

    # Conv Layer 5
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Conv5'))
    model.add(Dropout(0.2))

    # Pooling 2
    model.add(MaxPooling2D(pool_size=pool_size))

    # Upsample 1
    model.add(UpSampling2D(size=pool_size))

    # Deconv 1
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Deconv1'))
    model.add(Dropout(0.2))

    # Deconv 2
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Deconv2'))
    model.add(Dropout(0.2))

    # Upsample 2
    model.add(UpSampling2D(size=pool_size))

    # Deconv 3
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Deconv3'))
    model.add(Dropout(0.2))

    # Deconv 4
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Deconv4'))
    model.add(Dropout(0.2))

    # Deconv 5
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Deconv5'))
    model.add(Dropout(0.2))

    # Upsample 3
    model.add(UpSampling2D(size=pool_size))

    # Deconv 6
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Deconv6'))

    # Final layer - only including one channel so 1 filter
    model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'tanh', name = 'Final'))

    return model



In [ ]:

def main():
    # Load training images
    # train_images = pickle.load(open("/Users/divyamsobti10/Desktop/258/project/full_CNN_labels.p", "rb" ))
    train_images = pickle.load(open("/content/full_CNN_labels.p", "rb" ))

    # Load image labels
    # labels = pickle.load(open("/Users/divyamsobti10/Desktop/258/project/full_CNN_labels.p", "rb" ))
    labels = pickle.load(open("/content/full_CNN_labels.p", "rb" ))

    # Make into arrays as the neural network wants these
    train_images = np.array(train_images)
    labels = np.array(labels)

    # Normalize labels - training images get normalized to start in the network
    labels = labels / 255

    # Shuffle images along with their labels, then split into training/validation sets
    train_images, labels = shuffle(train_images, labels)
    # Test size may be 10% or 20%
    X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)

    # Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
    batch_size = 128
    epochs = 6
    pool_size = (2, 2)
    input_shape = X_train.shape[1:]

    # Create the neural network
    model = create_model(input_shape, pool_size)

    # Using a generator to help the model use less data
    # Channel shifts help with shadows slightly
    datagen = ImageDataGenerator(channel_shift_range=0.2)
    datagen.fit(X_train)

    # Compiling and training the model
    model.compile(optimizer='Adam', loss='mean_squared_error')
    model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
    epochs=epochs, verbose=1, validation_data=(X_val, y_val))

    # Freeze layers since training is done
    model.trainable = False
    model.compile(optimizer='Adam', loss='mean_squared_error')

    # Save model architecture and weights
    model.save('full_CNN_model.h5')

    # Show summary of model
    model.summary()

if __name__ == '__main__':
    main()


Epoch 1/10


<ipython-input-3-dacb18d754c8>:38: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,


89/89 [==============================] - 1267s 14s/step - loss: 0.0223 - val_loss: 0.0338
Epoch 2/10
89/89 [==============================] - 1220s 14s/step - loss: 0.0070 - val_loss: 0.0181
Epoch 3/10
89/89 [==============================] - 1222s 14s/step - loss: 0.0052 - val_loss: 0.0075
Epoch 4/10
89/89 [==============================] - 1212s 14s/step - loss: 0.0042 - val_loss: 0.0060
Epoch 5/10
89/89 [==============================] - 1211s 13s/step - loss: 0.0030 - val_loss: 0.0033
Epoch 6/10
89/89 [==============================] - 1215s 14s/step - loss: 0.0023 - val_loss: 0.0026
Epoch 7/10
89/89 [==============================] - 1217s 14s/step - loss: 0.0016 - val_loss: 0.0017
Epoch 8/10
89/89 [==============================] - 1213s 14s/step - loss: 0.0013 - val_loss: 0.0010
Epoch 9/10
89/89 [==============================] - 1217s 14s/step - loss: 0.0011 - val_loss: 0.0012
Epoch 10/10
76/89 [========================>.....] - ETA: 3:00 - loss: 9.9385e-04